# 20231212 - Pakkestorm
Jeg har vært på et temmelig hemmelig oppdrag og fulgt med på en server som har hatt mistenkelig oppførsel tidligere.  
Nå tok vi den igjen når den begynte å sende masse pakker, men selv om jeg som alle andre alver liker pakker så ble det litt for mye av det gode.  
Kan du finne de onde for meg?

[./data/20231212-fangede_pakker.pcap](./data/20231212-fangede_pakker.pcap)

In [1]:
import json
with open("./20231212/pakkestorm.json") as raw:
    packets = json.load(raw)

In [2]:
sessions = {}
for packet in packets:
    src  = int(packet["_source"]["layers"]["ip"]["ip.src_raw"][0], 16)
    dest = int(packet["_source"]["layers"]["ip"]["ip.dst_raw"][0], 16)
    data = int(packet["_source"]["layers"]["data_raw"][0], 16)
    raw  = int(packet["_source"]["layers"]["frame_raw"][0], 16)
    raw_len = packet["_source"]["layers"]["frame_raw"][2]
    session = {"src":src, "dst": dest, "data": data, "raw": raw, "raw_len":raw_len}
    if f"{src}->{dest}" not in sessions:
        sessions.update({f"{src}->{dest}":[session]})
    else:
        sessions[f"{src}->{dest}"].append(session)

In [15]:
for session in sessions:
    print(sessions[session][0]["src"], sessions[session][0]["dst"], str(sessions[session][0]["dst"]))

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x9b in position 1: invalid start byte

In [ ]:
for key, session in sessions.items():
    and_val = session[0]["raw"]
    bn_list = [bin(packet["raw"])[::-1] for packet in session]
    mask = []
    for index in range(0,len(max(bn_list))):
        mask.append("1")
        for bn in bn_list[2:]:
            if index >= len(bn):
                if bn_list[0][index] != "0":
                    mask.pop()
                    mask.append("0")
                    break
            if index < len(bn) and bn[index] != bn_list[0][index]:
                mask.pop()
                mask.append("0")
                break
    print("".join(mask))
    for packet in bn_list:
        print("".join([packet[index] if index < len(packet) and bitmask=="0" else " " for index,bitmask in enumerate(mask)]))
    #print()

In [ ]:
from string import ascii_lowercase, ascii_uppercase
strings = []
for key, session in sessions.items():
    string = "".join([chr(int(packet["data"].to_bytes(9).replace(b"\x00", b"")[2:], 2)) for packet in session])
    strings.append(string)
strings = list(set(strings))
strings.sort()
for string in strings:
    print(string)
for string in strings:
    if not [val for val in string[4:-2] if val in ascii_uppercase]:
        print(string)
        #print(string[:4]+"".join([val for val in string[4:-2] if val in ascii_uppercase])+"}")
    elif not [val for val in string[4:-2] if val in ascii_lowercase]:
        print(string)
        #print(string[4]+"".join([val for val in string[4:-2] if val in ascii_lowercase])+"}")